# ES7 Solution Sketch

:::{note}
Populate this portion with the problem prompt
:::


Model build notes (flow model):
- memory purge script DONE
- import MF6 FloPy script DONE
- define workspace script DONE
- set simulation name: DONE
- problem set up database: IN PROGRESS
- set flow model name: DONE
- set time discretization: DONE
- solver for flow model: DONE
- discretization for flow model: DONE
- initial conditions for flow model: DONE
- node property flow package: DONE
- storage package: DONE
- constant head package: DONE
- well package (if applicable): SKIP TRY 1
- output control for flow model: DONE
- Attempt FLOW model run: FAIL -- needs fixin'

In [57]:
# purge memory and reset_environment
import sys

def reset_namespace():
    """Delete all user-defined global variables."""
    globs = globals().copy()
    for name in globs:
        if not name.startswith("__") and not isinstance(globs[name], type(sys)):
            del globals()[name]

reset_namespace()

In [58]:
# import packages for modflow6 and flopy
import os
import pathlib as pl
from pprint import pformat
import flopy
#import git
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
from flopy.plot.styles import styles
#from modflow_devtools.misc import get_env, timed
import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)

In [59]:
# define workspace and executibles -- THISISMACHINE SPECIFIC
binary = "/home/sensei/mfplayground/modflow-python/mf6.4.1_linux/bin/mf6"  # location on MY computer of the compiled modflow program
directory_name = "es7_soln"
workarea = os.path.join("/home/sensei/ce-5364-webroot/mfexperiments", directory_name)
os.makedirs(workarea, exist_ok=True)

In [60]:
# set simulation name(s)
name = "es7"
gwfname = "gwf-" + name
gwtname = "gwt-" + name

##### FLOPY build simulation framework ####
sim = flopy.mf6.MFSimulation(
    sim_name="sim-" + name, exe_name=binary, version="mf6", sim_ws=workarea
)
# set model name (using same base name as the simulation)
model_nam_file = "{}.nam".format(gwfname)
# create MODFLOW6 flow model framework
gwf = flopy.mf6.ModflowGwf(sim, modelname=gwfname, save_flows=True, model_nam_file=model_nam_file)
## delete ";" in above line at end to show full output

In [61]:
#Problem set-up database and simulation control constants

# Console output flags
verbose = True # Used for tree-killing level of output; set to False for production run(s)

# Model units
length_units = "meters"
time_units = "seconds"

# Model grid geometry
nlay = 3  # Number of layers
nrow = 50  # Number of rows
ncol = 26  # Number of columns
# Layer 1 == top, Layer 2== middle, Layer 3 ==bottom
delr = 200.0*(1.0/3.28)  # Column width ($m$)
delc = 200.0*(1.0/3.28)  # Row width ($m$)
delz = [175.0*(1.0/3.28),15.0*(1.0/3.28),60.0*(1.0/3.28)]  # Layer thickness ($m$)
top = 3150.0*(1.0/3.28)  # Top of the model ($m$)
bottom = np.array([2975.0*(1.0/3.28),2960.0*(1.0/3.28),2900.0*(1.0/3.28)]) # layer bottom elevations


# Aquifer hydraulic properties
pors1 = 0.38
pors2 = 0.40
pors3 = 0.35
porosity = [pors1,pors2,pors3]  # Porosity
k1 = 25.0*(1.0/3.28)  # Horiz. hyd. conductivity of top layer ($m/sec$)
k2 = 0.25*(1.0/3.28)  # Horiz. hyd. conductivity of middlelayer ($m/sec$)
k3 = 30.0*(1.0/3.28)  # Horiz. hyd. conductivity of bottom layer ($m/sec$)
hydcon = [k1,k2,k3]
stor1 = 0.20000
stor2 = 0.00040
stor3 = 0.00035
storativity = [stor1,stor2,stor3]

# Simulation layer types
laytyp = icelltype = 0  # this will need modification later on.  laytyp may need to be a list

# Aquifer transport properties
al = 10.0*(1/3.28) # Longitudinal dispersivity ($m$)
at =  1.0*(1/3.28) # Transverse dispersivity ($m$) 
### Assume dispeasuvuty same all layers
trpt = at/al  # Ratio of horiz. transverse to longitudinal dispersivity ($m$)
### If adsorbtion
kd = 0.20
### If 1st order decay
lamb = 1.0


# build porosity array(s)
pors_array = np.ones((nlay, nrow, ncol), dtype=float)
for ilay in range(nlay):
    pors_array[ilay] = porosity[ilay]*pors_array[ilay]
if verbose:
    print("\n porosity array(s)")
    for ilay in range(nlay):
        print(f"\nLayer {ilay + 1}")
        print("-" * (4 + 3 * ncol))  # simple header line
        for irow in range(nrow):
            print(" ".join(f"{val:.2f}" for val in pors_array[ilay, irow, :]))

# build hydraulic conductivity array(s)
hk_array = np.ones((nlay, nrow, ncol), dtype=float)
for ilay in range(nlay):
    hk_array[ilay] = hydcon[ilay]*hk_array[ilay]
if verbose:
    print("\n hydraulic conductivity array(s)")
    for ilay in range(nlay):
        print(f"\nLayer {ilay + 1}")
        print("-" * (4 + 3 * ncol))  # simple header line
        for irow in range(nrow):
            print(" ".join(f"{val:.2f}" for val in hk_array[ilay, irow, :]))

# build storativity array(s)
ss_array = np.ones((nlay, nrow, ncol), dtype=float)
for ilay in range(nlay):
    ss_array[ilay] = storativity[ilay]*ss_array[ilay]
if verbose:
    print("\n storage coefficient array(s)")
    for ilay in range(nlay):
        print(f"\nLayer {ilay + 1}")
        print("-" * (4 + 3 * ncol))  # simple header line
        for irow in range(nrow):
            print(" ".join(f"{val:.4f}" for val in ss_array[ilay, irow, :]))
            
# build computation domain array(s)
idomain = np.ones((nlay, nrow, ncol), dtype=int)  # entire 3d domain
if verbose:
    print("\n computation domain array(s)")
    for ilay in range(nlay):
        print(f"\nLayer {ilay + 1}")
        print("-" * (4 + 3 * ncol))  # simple header line
        for irow in range(nrow):
            print(" ".join(f"{val:3d}" for val in idomain[ilay, irow, :]))

# build boundary array(s) -- assume boundary conditions to all layers
# Boundaries
# Create a 3D array of zeros
ibound = np.zeros((nlay, nrow, ncol), dtype=int) # entire 3d domain
for ilay in range(nlay):
    ibound[ilay, 0, 0:21] = [-1 for i in range(ncol-5)] # top row all -1
    ibound[ilay,-1, 9:] = [-1 for i in range(9,ncol)] # bottom row all -1
    ##########################
    ibound[ilay,1,0]= -1 
    ibound[ilay,2,0]= -1 
    ibound[ilay,3,0]= -1 
    ##########################
    ibound[ilay,-2,-1]= -1 
    ibound[ilay,-3,-1]= -1 
    ibound[ilay,-4,-1]= -1 
    ibound[ilay,-5,-1]= -1 
if verbose:
    print("boundary condition array(s)")
#    print(ibound)
    for ilay in range(nlay):
        print(f"\nLayer {ilay + 1}")
        print("-" * (4 + 3 * ncol))  # simple header line
        for irow in range(nrow):
            print(" ".join(f"{val:3d}" for val in ibound[ilay, irow, :]))

# Build initial conditions array(s)
# Create a 3D array of ones
inithead = (0.5*(3100+3040))*np.ones((nlay, nrow, ncol), dtype=float) # entire 3d domain set to 1.0
for ilay in range(nlay):
    inithead[ilay, 0,:] = [3100.0 for i in range(ncol)] # top row all -1
    inithead[ilay,-1,:] = [3040.0 for i in range(ncol)] # bottom row all -1
    ##########################
    inithead[ilay,1,0]= 3100.0
    inithead[ilay,2,0]= 3100.0 
    inithead[ilay,3,0]= 3100.0
    ##########################
    inithead[ilay,-2,-1]= 3040.0 
    inithead[ilay,-3,-1]= 3040.0 
    inithead[ilay,-4,-1]= 3040.0  
    inithead[ilay,-5,-1]= 3040.0 
if verbose:
    print("initial head condition array(s)")
#    print(ibound)
    for ilay in range(nlay):
        print(f"\nLayer {ilay + 1}")
        print("-" * (4 + 3 * ncol))  # simple header line
        for irow in range(nrow):
            print(" ".join(f"{val:.0f}" for val in inithead[ilay, irow, :]))

# Build dispersivity arrays

al = 10.0*(1/3.28) # Longitudinal dispersivity ($m$)
trpt = 0.1  # Ratio of horiz. transverse to longitudinal dispersivity ($m$)

# Build distribution coefficient arrays
kd_b = 0.20

# Build 1st order decay arrays

decay_lambda = 0.00

# Build time information

# Time variables
perlen = [365.0 * 86400, 365.0 * 86400]
steady = [False, False]
nper = len(perlen)
nstp = [365, 365]
tsmult = [1.0, 1.0]

# Solver settings
nouter, ninner = 100, 300
hclose, rclose, relax = 1e-6, 1e-6, 1.0
percel = 1.0  # HMOC parameters
itrack = 2
wd = 0.5
dceps = 1.0e-5
nplane = 0
npl = 0
nph = 16
npmin = 2
npmax = 32
dchmoc = 1.0e-3
nlsink = nplane
npsink = nph
nadvfd = 1


 porosity array(s)

Layer 1
----------------------------------------------------------------------------------
0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38
0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38
0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38
0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38
0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38
0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38
0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.38 0.3

The workflow for the flow model will generally involve the following:
```
###### Instantiating MODFLOW 6 groundwater flow model            ########
###### Instantiating MODFLOW 6 time discretization               ########
###### Instantiating MODFLOW 6 solver for flow model             ########
###### Instantiating MODFLOW 6 discretization for flow model     ########
###### Instantiating MODFLOW 6 initial conditions for flow model ########
###### Instantiating MODFLOW 6 node-property flow package        ########
###### Instantiating MODFLOW 6 storage package                   ########
###### Instantiating MODFLOW 6 constant head package             ########
###### Instantiating MODFLOW 6 well package (if applicable)      ########
###### Instantiating MODFLOW 6 output control for flow model     ########
```
These are created one-by-one in the cells below

In [62]:
###### Instantiating MODFLOW 6 groundwater flow model            ########
# Set Model Name (using same base name as the simulation)
model_nam_file = "{}.nam".format(gwfname)
# create MODFLOW6 flow model framework
if verbose:
    gwf = flopy.mf6.ModflowGwf(sim, modelname=gwfname, save_flows=True, model_nam_file=model_nam_file)
    print("flow model framework created into object: ""gwf"" ")
else:
    gwf = flopy.mf6.ModflowGwf(sim, modelname=gwfname, save_flows=True, model_nam_file=model_nam_file);

flow model framework created into object: gwf 


In [63]:
###### Instantiating MODFLOW 6 time discretization               ########
tdis_rc = []
for i in range(nper):
    tdis_rc.append((perlen[i], nstp[i], tsmult[i]))
if verbose:
    flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_rc, time_units=time_units)
    print("time discritization framework created into object: ""tdis_rc"" ")
else:
    flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_rc, time_units=time_units);

time discritization framework created into object: tdis_rc 


In [64]:
###### Instantiating MODFLOW 6 solver for flow model             ########
# Set Iterative Model Solution (choose solver parameters)
# about IMS see: https://water.usgs.gov/nrp/gwsoftware/ModelMuse/Help/sms_sparse_matrix_solution_pac.htm
# using defaults see: https://flopy.readthedocs.io/en/3.3.2/source/flopy.mf6.modflow.mfims.html
imsgwf = flopy.mf6.ModflowIms(
    sim,
    print_option="SUMMARY",
    outer_dvclose=hclose,
    outer_maximum=nouter,
    under_relaxation="NONE",
    inner_maximum=ninner,
    inner_dvclose=hclose,
    rcloserecord=rclose,
    linear_acceleration="CG",
    scaling_method="NONE",
    reordering_method="NONE",
    relaxation_factor=relax,
    filename=f"{gwfname}.ims",
    )
if verbose:
    sim.register_ims_package(imsgwf, [gwf.name])
    print("solver framework created into object: ""imsgwf"" ")
else:
    sim.register_ims_package(imsgwf, [gwf.name]);   

solver framework created into object: imsgwf 


In [65]:
###### Instantiating MODFLOW 6 discretization for flow model     ########
if verbose:
    flopy.mf6.ModflowGwfdis(
    gwf,
    length_units=length_units,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=bottom,
    idomain=idomain,
    filename=f"{gwfname}.dis",
    )
    print("grid framework created into object: ""name.dis"" ")
else:
    flopy.mf6.ModflowGwfdis(
    gwf,
    length_units=length_units,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=bottom,
    idomain=idomain,
    filename=f"{gwfname}.dis",
    );


grid framework created into object: name.dis 


In [66]:
###### Instantiating MODFLOW 6 initial conditions for flow model ########
if verbose:
    flopy.mf6.ModflowGwfic(gwf, strt=inithead, filename=f"{gwfname}.ic")
    print("initial heads framework created into object: ""name.ic"" ")
else:
    flopy.mf6.ModflowGwfic(gwf, strt=inithead, filename=f"{gwfname}.ic");

initial heads framework created into object: name.ic 


In [67]:
###### Instantiating MODFLOW 6 node-property flow package        ########
if verbose:
    flopy.mf6.ModflowGwfnpf(
    gwf,
    save_flows=False,
    icelltype=icelltype,
    k=hk_array,
    k33=hk_array,
    save_specific_discharge=True,
    filename=f"{gwfname}.npf",
    )
    print("node-property framework created into object: ""name.npf"" ")
else:
    flopy.mf6.ModflowGwfnpf(
    gwf,
    save_flows=False,
    icelltype=icelltype,
    k=hk_array,
    k33=hk_array,
    save_specific_discharge=True,
    filename=f"{gwfname}.npf",
    );

node-property framework created into object: name.npf 


In [76]:
###### Instantiating MODFLOW 6 storage package                   ########
if verbose:
    sto = flopy.mf6.ModflowGwfsto(gwf,storagecoefficient=True, ss=ss_array)
    print("storage framework created into object: ""sto"" ")
else:
    sto = flopy.mf6.ModflowGwfsto(gwf,storagecoefficient=True, ss=ss_array);
## delete ";" in above line at end to show full output

storage framework created into object: sto 


In [77]:
###### Instantiating MODFLOW 6 constant head package             ########
if verbose:
    chdspd = []
    for ilay in range(nlay):
        for irow in range(nrow):
            for jcol in range(ncol):
                if ibound[ilay,irow,jcol]==-1 and irow <= 6:
                    chdspd.append([(ilay, irow, jcol), 3100.0, 0.0]) # set head and concentration
                if ibound[ilay,irow,jcol]==-1 and irow >= 44:
                    chdspd.append([(ilay, irow, jcol), 3040.0, 0.0]) # set head and concentration              
    flopy.mf6.ModflowGwfchd(
    gwf,
    maxbound=len(chdspd),
    stress_period_data=chdspd,
    save_flows=False,
    auxiliary="CONCENTRATION",
    pname="CHD-1",
    filename=f"{gwfname}.chd",
    )
    print("storage framework created into object: ""chdspd"" ")
else:
    chdspd = []
    for ilay in range(nlay):
        for irow in range(nrow):
            for jcol in range(ncol):
                if ibound[ilay,irow,jcol]==-1 and irow <= 6:
                    chdspd.append([(ilay, irow, jcol), 3100.0, 0.0]) # set head and concentration
                if ibound[ilay,irow,jcol]==-1 and irow >= 44:
                    chdspd.append([(ilay, irow, jcol), 3040.0, 0.0]) # set head and concentration              
    flopy.mf6.ModflowGwfchd(
    gwf,
    maxbound=len(chdspd),
    stress_period_data=chdspd,
    save_flows=False,
    auxiliary="CONCENTRATION",
    pname="CHD-1",
    filename=f"{gwfname}.chd",
    );
## delete ";" in above line at end to show full output

storage framework created into object: chdspd 


In [70]:
###### Instantiating MODFLOW 6 well package (if applicable)      ########
if verbose:
    print("no wells this simulation")
else:
    pass

no wells this simulation


In [71]:
###### Instantiating MODFLOW 6 output control for flow model     ########
if verbose:
    flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=f"{gwfname}.hds",
    budget_filerecord=f"{gwfname}.bud",
    headprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
    saverecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
    printrecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
    )
    print("output control for flow model created into objects ""name.hds"" and ""name.bud"" ")
else:
    flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=f"{gwfname}.hds",
    budget_filerecord=f"{gwfname}.bud",
    headprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
    saverecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
    printrecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
    );

output control for flow model created into objects name.hds and name.bud 


In [78]:
# Write files to the directory - FLOW MODEL ONLY
if verbose:
    sim.write_simulation()
else:
    sim.write_simulation();

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...
  writing model gwf-es7...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package oc...
    writing package sto...
    writing package chd-1...
